## Init e dependencies

In [ ]:
! pip install langchain==0.2.7 langchain_aws langchain-community langchain_core boto3 botocore
! pip install dotenv

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [64 lines of output]
      /usr/lib/python3/dist-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
        warnings.warn(
        error: subprocess-exited-with-error
      
        × python setup.py egg_info did not run successfully.
        │ exit code: 1
        ╰─> [16 lines of output]
            Traceback (most recent call last):
              File "<string>", line 2, in <module>
              File "<pip-setuptools-caller>", line 14, in <module>
              File "/tmp/pip-wheel-30hirj05/distribute_fd6c7cc820a74ea0b7b

In [34]:
! pip install pymupdf python-docx pydantic

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from dotenv import load_dotenv
load_dotenv()
import os

### llm endpoints (to edit)

In [ ]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import urllib3
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name='eu-west-1',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        verify=False,  # Disable SSL verification
        config=Config(
            proxies={'https': None}
        )
    )

llm = ChatBedrock(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        client=bedrock_client,
        model_kwargs={
            "temperature": 0,
            "max_tokens": 2000,
        }
    )


In [28]:
import fitz  # PyMuPDF
import docx

def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([p.text for p in doc.paragraphs])
    return text

def read_txt(file_path):
    with open(file_path, "r") as f:
        text = f.read()
    return text



In [21]:
def generate(text):

    prompt_template = PromptTemplate(
        input_variables=["protagonist", "setting"],
        template="""
            Riassumi il seguente testo:
            {text}
            
            REGOLE
            - scrivi una breve intro per spiegare il contesto
            - riassumi i punti principali in una bullet list
            - trai le tue conclusioni sull'argomento
        """
    )

    chain = prompt_template | llm | StrOutputParser()

    response = chain.invoke({"text": text})
    return response


In [29]:
file_path = "RomeCall_Paper_web.pdf"  # Se è nella stessa cartella del notebook

text = read_pdf(file_path)

response = generate(text)

In [31]:
from IPython.display import display, Markdown
display(Markdown(f"### Riassunto:\n\n{response}"))
    

### Riassunto:

Introduzione
Il testo presenta la "Rome Call for AI Ethics", un documento firmato nel 2020 per promuovere un approccio etico all'intelligenza artificiale. L'obiettivo è di creare un futuro in cui l'innovazione digitale e il progresso tecnologico mettano l'essere umano al centro.

Punti principali:
- L'IA offre grandi potenzialità per migliorare il benessere sociale e personale, ma deve essere sviluppata nel rispetto della dignità umana e senza avere come unico scopo il profitto.
- I firmatari si impegnano a promuovere uno sviluppo dell'IA al servizio dell'intera umanità, che non discrimini nessuno e che sia attento all'ambiente.
- Vengono definiti 6 principi etici fondamentali per l'IA: trasparenza, inclusione, responsabilità, imparzialità, affidabilità, sicurezza e privacy.
- Viene sottolineata l'importanza dell'istruzione e della formazione per sensibilizzare sulle opportunità e sui rischi dell'IA.
- Sono necessarie nuove forme di regolamentazione per promuovere la trasparenza e il rispetto dei principi etici, soprattutto per le tecnologie ad alto rischio di impatto sui diritti umani.

Conclusioni
La "Rome Call for AI Ethics" rappresenta un importante passo avanti nella definizione di un quadro etico per lo sviluppo dell'intelligenza artificiale, con l'obiettivo di garantire che il progresso tecnologico sia al servizio dell'umanità e del pianeta. L'impegno congiunto di organizzazioni internazionali, governi, istituzioni e aziende tecnologiche è fondamentale per realizzare questa visione.

In [41]:
file_path = "docs/football_league.docx"

text = read_docx(file_path)

summary = generate(text)

display(Markdown(f"### Riassunto:\n\n{response}"))

### Riassunto:

Introduzione:
Il report annuale Deloitte Football Money League analizza le performance finanziarie dei principali club di calcio a livello globale, evidenziando una continua crescita del settore.

Punti principali:
- I primi 20 club per fatturato hanno registrato ricavi complessivi pari a 11,2 miliardi di euro, segnando un incremento del +6% rispetto alla stagione precedente.
- Il Real Madrid è diventato la prima squadra nella storia a superare 1 miliardo di euro di entrate in una singola stagione.
- I ricavi commerciali hanno raggiunto 4,9 miliardi di euro, confermandosi la principale fonte di entrate.
- I ricavi da matchday sono cresciuti del +11%, toccando per la prima volta quota 2 miliardi di euro.
- Il successo sul campo è un fattore determinante per la crescita economica dei club.
- Il calcio femminile ha registrato una forte espansione, superando 100 milioni di euro complessivi di entrate.

Conclusioni:
Il report evidenzia la continua crescita del settore calcistico a livello globale, trainata principalmente dai ricavi commerciali e dai proventi da matchday. Il successo sportivo rimane un fattore chiave per la performance economica dei club, mentre il calcio femminile sta guadagnando sempre più rilevanza nel panorama finanziario del calcio.

In [50]:
from pydantic import BaseModel
from langchain.output_parsers import PydanticOutputParser

class IndicatorData(BaseModel):
    indicator: str
    value: str

class IndicatorDataList(BaseModel):
    data: list[IndicatorData]


parser = PydanticOutputParser(pydantic_object=IndicatorDataList)

def extract_summary_data(text):
    prompt_template = PromptTemplate(
        input_variables=["text"],
        template="""
            Estrai i dati chiave dal seguente testo .
            {text}
            
            ISTRUZIONI
            {format_instructions}
        """,
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt_template | llm | parser
    
    response = chain.invoke({"text": text})
    
    # try:
    #     data_dict = response  # JsonOutputParser dovrebbe già restituire un dizionario
    #     data = [IndicatorData(indicator=k, value=v) for k, v in data_dict.items()]
    # except Exception:
    #     data = [IndicatorData(indicator="errore", value="Impossibile estrarre dati strutturati")]
    
    return response
    
summary_data = extract_summary_data(summary)

import json

display(Markdown(f"### Riassunto:\n\n{summary}"))

display(summary_data)

    


TypeError: 'ModelMetaclass' object is not iterable